In [2]:
import csv
from datetime import datetime

In [3]:
# number of seconds since 8am
def time_unit(timestamp):
    hh_mm_ss = timestamp.split(' ')[1]
    arr = hh_mm_ss.split(':')
    h = int(arr[0])
    m = int(arr[1])
    s = int(arr[2])
    return s + m*60 + (h-8)*60*60

Want: one csv file in the form

```id, [x @ ts1, y @ ts1, RIDE?], [x @ ts2, y @ ts2, RIDE?], ..., [x @ ts57600, y @ ts57600, RIDE?]```

Need to consider what the loc should be if the person is on a ride.
Also, it looks like the movement csv files are sorted by timestamp, which is super helpful.

In [4]:
movement_data_prefix = './hw02-data/Movement Data/park-movement'
day = 'Fri'
movement_data_path = f'{movement_data_prefix}-{day}.csv'

In [5]:
# first, gather all of the possible ids by traversing the csv file once
id_dict = {}
with open(movement_data_path) as movement_data_csv:
    reader = csv.reader(movement_data_csv)
    first_row = True
    for row in reader:
        if first_row:
            first_row = False
            continue
        id = row[1]
        id_dict[id] = True

In [36]:
len(id_dict.keys())

3557

In [11]:
movement_data_by_id_path = f'{movement_data_prefix}-{day}-by-id.csv'

# new movement data file to write to
with open(movement_data_by_id_path, 'a', newline='') as movement_data_by_id_csv:
    writer = csv.writer(movement_data_by_id_csv)

    # original movement data to read from
    with open(movement_data_path) as movement_data_csv:
        # row in format [Timestamp,id,type,X,Y]
        reader = csv.reader(movement_data_csv)

        # create movement data based on id
        for id in id_dict.keys():
            id_row = [id]
            id_row.extend([None]*16*60*60)
            first_row = True
            if not first_row:
                exit
            for row in reader:
                if first_row:
                    first_row = False
                    continue
                # check if the id matches
                row_id = row[1]
                if not id == row_id:
                    continue
                row_timestamp = row[0]
                row_time_unit = time_unit(row_timestamp)
                row_x = row[3]
                row_y = row[4]
                row_type = row[2]
                id_row[row_time_unit + 1] = [row_x, row_y, row_type]
                
            # if last time was 'check in' and then there's anything after it, record as same location and put type='ride'
            # if last time was 'movement' and there's anything after it, record as same location and put type='stopped'
            i = 1
            # move to first non-None cell
            while id_row[i] is None and i < len(id_row)-1:
                i += 1

            while i < len(id_row)-1:
                # ensure this isn't just the last timestamp that they were in the park
                j = i+1
                left_park = False
                while id_row[j] is None:
                    j += 1
                    if j >= len(id_row) - 1:
                        left_park = True
                        break

                # if they were in the park the whole time, fill in the missing time stamps with either 'ride' or 'stop' location, inferred from last seen timestamp
                if not left_park:
                    j = i + 1
                    while id_row[j] is None:
                        id_row[j] = [id_row[i][0], id_row[i][1], None]
                        if id_row[i][2] == 'check-in':
                             id_row[j][2] = 'ride'
                        elif id_row[i][2] == 'movement':
                             id_row[j][2] = 'stop'
                        j += 1      
                i = j
            
            # finally, write the row into the csv file
            written_row = [id]
            first_pos = True
            for loc in id_row:
                if first_pos:
                    first_pos = False
                    continue
                if loc is None:
                    written_row.extend([None, None, None])
                else:
                    written_row.append(loc[0])
                    written_row.append(loc[1])
                    written_row.append(loc[2])
            writer.writerow(written_row)



Want: another csv file in the form

``timestamp, id, x, y, onride?``

* basically, the same as the park-movement csv files, except every timestamp for every id